In [1]:
%reset -f

import glob
import os
import pandas as pd
import datetime as dt
import numpy as np
import datetime as dt

import pandas as pd
import os
import glob
import numpy as np
import datetime as dt
from google.cloud import bigquery

###
import config_utility
credential_path = config_utility.gbq_credential_path
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

today = str(dt.datetime.today()).split(' ')[0]
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

def list_files_in_directory(path):
    '''docstring for list_files_in_directory'''
    x = []
    cwd = os.getcwd()
    for root, dirs, files in os.walk('.'+path):
        for file in files:
            #print(root+'/'+file)
            x.append(root+'/'+file)
    return x

def display_file_list(files):
    '''docstring for display_file_list'''
    for file in files:
        print(file)
        try:
            df = pd.read_csv(file)
            print(len(df))
            display(df.head())
        except EmptyDataError as e:
            print(e)
    return

def download_table_from_gbq(project_name, dataset_name, table_name):
    '''docstring for download_table_from_gbq'''
    # https://cloud.google.com/bigquery/docs/pandas-gbq-migration
    client = bigquery.Client()
    sql = """
        SELECT *
        FROM `{project}.{dataset}.{table}`
        """
    query = sql.format(project=project_name, dataset=dataset_name, table=table_name)
    # Run a Standard SQL query using the environment's default project
    df = client.query(sql).to_dataframe()
    # Run a Standard SQL query with the project set explicitly
    project_id = 'peronal-data-projects'
    df = client.query(sql, project=project_id).to_dataframe()
    return df

def upload_table_to_gbq(df, dataset_name, table_name):
    '''docstring for upload_table_to_gbq'''
    #https://cloud.google.com/bigquery/docs/pandas-gbq-migration
    client = bigquery.Client()
    table_id = dataset_name+'.'+table_name
    job = client.load_table_from_dataframe(df, table_id)
    # Wait for the load job to complete.
    print('job complete')
    return job.result()

# ###
# # download table from google bigquery script_uploads dataset
# project_name = 'peronal-data-projects'
# dataset_name = 'script_uploads'
# table_name = 'raw_twl_customer_table'
# df = download_table_from_gbq(project_name, dataset_name, table_name)

# ###
# # upload dataframe to google bigquery script_uploads dataset
# dataset_name = 'script_uploads'
# table_name = 'raw_recurring_spend_tracking_sheet'
# upload_table_to_gbq(ndf, dataset_name, table_name)

###
# create list of files in directory
path = '/lightspeed'
x = list_files_in_directory(path)
# select for csvs
files = [i for i in x if 'csv' in i]
# [i for i in files if 'transaction' in i]
# select for inventory table
items = [i for i in files if 'item' in i][0]
# dataframe from local file
df = pd.read_csv(items)
cols = [i.replace(' ','_').replace('.','').lower() for i in list(df)]
df.columns = cols

###
# upload dataframe to google bigquery script_uploads dataset
dataset_name = 'script_uploads'
table_name = 'raw_twl_customer_table'
# upload_table_to_gbq(df, dataset_name, table_name)

In [9]:
# sales = [i for i in files if 'transaction' in i]
# display_file_list(sales)

filename = './lightspeed/csvs/sales/2020-07-12_reports_sales_listings_transaction_line(3).csv'
df = pd.read_csv(filename)

df0 = pd.read_csv(items)
df1 = df0.drop_duplicates(subset = 'Item', keep = 'first', inplace = False)

ndf = df.merge(df1,left_on='Description',right_on='Item',how='left')
cols = [i.replace(' ','_').replace('.','').lower() for i in list(ndf)]
ndf.columns = cols

###
# upload dataframe to google bigquery script_uploads dataset
dataset_name = 'script_uploads'
table_name = 'base_twl_sales_with_item_info'
# upload_table_to_gbq(ndf, dataset_name, table_name)

In [3]:
dfs = [pd.DataFrame(ndf.dtypes).T
       , pd.DataFrame(ndf.count()).T]

temp = pd.concat(dfs).T
temp.columns = ['type','counts']
l = list(temp.loc[temp['type']=='float64'].index)
for i in l:
    print(i)
    for j in ndf[i]:
        #print(j)
        try:
            if '<' in j:
                print(i,'---',j)
        except TypeError as e:
            pass

work_order_internal_note
system_id
upc
ean
season
department
msrp
default_cost
subcategory_1
subcategory_2
subcategory_3
subcategory_4
subcategory_5
subcategory_6
subcategory_7
subcategory_8
subcategory_9


In [10]:
table_name = 'base_twl_sales_with_item_info'
ndf.to_csv(today+'_'+table_name+'.csv',index=False)


In [33]:
# ndf

In [32]:
# ndf.loc[ndf['category']=='Disc']#.loc[ndf['qty']]
# # remove second qty

In [30]:
# converting flat txt file to csv

test = pd.read_fwf([i for i in x if 'txt' in i][0])
# pd.read_csv([i for i in x if 'txt' in i][0], sep="  ", header=None)

cols = list(test)[0].split('\t')
data = list(test[list(test)[0]])
data_list = []
for row in data:
    data_list.append(row.split('\t'))
    
test2 = pd.DataFrame(data_list)
# display(test2)
test2.columns = cols
filename = [i for i in x if 'txt' in i][0].replace('txt','csv')
test2.to_csv(filename,index=False)

In [31]:
# [i for i in x if 'txt' in i][0].replace('txt','csv')